In [28]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
# sc= SparkContext()
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('s3://aws-sa-ray-us-east-1/mydata.csv')
df.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(x1=1, x2=147, x3=68)]

In [29]:
df.cache()
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- x1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- x3: integer (nullable = true)

In [30]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version   
-------------------------- ----------
beautifulsoup4             4.9.3     
boto                       2.49.0    
click                      7.1.2     
cycler                     0.10.0    
jmespath                   0.10.0    
joblib                     0.17.0    
kiwisolver                 1.3.1     
lxml                       4.6.1     
matplotlib                 3.4.1     
mysqlclient                1.4.2     
nltk                       3.5       
nose                       1.3.4     
numpy                      1.16.5    
pandas                     1.2.3     
Pillow                     8.1.2     
pip                        9.0.1     
py-dateutil                2.2       
pyparsing                  2.4.7     
python-dateutil            2.8.1     
python37-sagemaker-pyspark 1.4.1     
pytz                       2020.1    
PyYAML                     5.3.1     
regex                      2020.10.28
setuptools                 28.8.0    
six         

In [31]:
sc.install_pypi_package("pandas") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [32]:
df.describe().toPandas().transpose()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

               0           1                   2    3        4
summary    count        mean              stddev  min      max
x1       1000000    500000.5   288675.2789323441    1  1000000
x2       1000000  159.979326  17.607377221081556  130      190
x3       1000000   80.011905  23.373147061852208   40      120

In [42]:
import six
for i in df.columns:
    print( "Correlation to x3 for ", i, df.stat.corr('x3',i))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correlation to x3 for  x1 0.00014978178757055975
Correlation to x3 for  x2 0.0009398301078493216
Correlation to x3 for  x3 1.0

In [43]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['x1','x2'], outputCol = 'features')
v_df = vectorAssembler.transform(df)
v_df = v_df.select(['features', 'x3'])
v_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---+
|   features| x3|
+-----------+---+
|[1.0,147.0]| 68|
|[2.0,144.0]| 96|
|[3.0,167.0]|104|
+-----------+---+
only showing top 3 rows

In [44]:
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='x3', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coefficients: [0.0,0.0]
Intercept: 79.99309421061801

In [46]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE: 23.375243
r2: 0.000000

In [47]:
train_df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|               x3|
+-------+-----------------+
|  count|           700282|
|   mean|79.99309421061801|
| stddev| 23.3752597365512|
|    min|               40|
|    max|              120|
+-------+-----------------+

In [50]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","x3","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="x3",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---+-----------+
|       prediction| x3|   features|
+-----------------+---+-----------+
|79.99309421061801| 68|[1.0,147.0]|
|79.99309421061801| 96|[2.0,144.0]|
|79.99309421061801|104|[3.0,167.0]|
|79.99309421061801|118|[5.0,163.0]|
|79.99309421061801| 81|[6.0,183.0]|
+-----------------+---+-----------+
only showing top 5 rows

R Squared (R2) on test data = -7.21339e-06

In [51]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 23.3682

In [52]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

numIterations: 1
objectiveHistory: [0.5]
+--------------------+
|           residuals|
+--------------------+
| -14.993094210618011|
|   25.00690578938199|
| -15.993094210618011|
|   25.00690578938199|
|   32.00690578938199|
|0.006905789381988825|
|  -22.99309421061801|
|  -7.993094210618011|
|   5.006905789381989|
| -15.993094210618011|
|   26.00690578938199|
|  -7.993094210618011|
|   2.006905789381989|
|  12.006905789381989|
|0.006905789381988825|
|  -27.99309421061801|
|  -9.993094210618011|
|  11.006905789381989|
|   31.00690578938199|
|   9.006905789381989|
+--------------------+
only showing top 20 rows

In [54]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","x3","features").show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---+------------+
|       prediction| x3|    features|
+-----------------+---+------------+
|79.99309421061801| 68| [1.0,147.0]|
|79.99309421061801| 96| [2.0,144.0]|
|79.99309421061801|104| [3.0,167.0]|
|79.99309421061801|118| [5.0,163.0]|
|79.99309421061801| 81| [6.0,183.0]|
|79.99309421061801| 68| [8.0,142.0]|
|79.99309421061801| 40|[13.0,162.0]|
|79.99309421061801| 61|[16.0,142.0]|
|79.99309421061801|102|[20.0,159.0]|
|79.99309421061801| 96|[22.0,174.0]|
|79.99309421061801| 50|[23.0,182.0]|
|79.99309421061801| 40|[25.0,140.0]|
|79.99309421061801| 40|[29.0,178.0]|
|79.99309421061801| 64|[35.0,190.0]|
|79.99309421061801| 61|[38.0,160.0]|
|79.99309421061801| 61|[39.0,173.0]|
|79.99309421061801| 74|[40.0,155.0]|
|79.99309421061801|107|[42.0,172.0]|
|79.99309421061801|113|[43.0,151.0]|
|79.99309421061801| 47|[44.0,159.0]|
+-----------------+---+------------+
only showing top 20 rows